# Intro to Neural Networks
- Course: Deep Learning Foundation Nanodegree
- Lesson 7: Intro to Neural Networks
- Topic: Gradient Descent 

The example below shows the implementation of Gradiant Descent into a Neural Network with a single neuron in order to update the weights. Check topic 12 for algorithm description for gradient application on a dataset (instead of only one input dataset as previous example). A new error calculation for threshold is also introduced (Mean of Square Errors).

Dataset context: University admission based on GPA, GRE and School rank.
http://www.ats.ucla.edu/stat/data/binary.csv.

The following section prepares the input data for a gaussian distribution 0 centered with standard deviation 1.

In [1]:
import numpy as np
import pandas as pd

admissions = pd.read_csv('binary.csv')

# Make dummy variables for rank
data = pd.concat([admissions, pd.get_dummies(admissions['rank'], prefix='rank')], axis=1)
data = data.drop('rank', axis=1)

# Standarize features
for field in ['gre', 'gpa']:
    mean, std = data[field].mean(), data[field].std()
    data.loc[:,field] = (data[field]-mean)/std
    
# Split off random 10% of the data for testing
np.random.seed(42)
sample = np.random.choice(data.index, size=int(len(data)*0.9), replace=False)
data, test_data = data.ix[sample], data.drop(sample)

# Split into features and targets
features, targets = data.drop('admit', axis=1), data['admit']
features_test, targets_test = test_data.drop('admit', axis=1), test_data['admit']


Apply Gradient Descent to optimize weights

In [11]:
import numpy as np
#from data_prep import features, targets, features_test, targets_test


def sigmoid(x):
    """
    Calculate sigmoid
    """
    return 1 / (1 + np.exp(-x))

# Use to same seed to make debugging easier
np.random.seed(42)

n_records, n_features = features.shape
last_loss = None

# Initialize weights
weights = np.random.normal(scale=1 / n_features**.5, size=n_features)

# Neural Network hyperparameters
epochs = 1000
learnrate = 0.05

for e in range(epochs):
    del_w = np.zeros(weights.shape)
    for x, y in zip(features.values, targets):
        
        # Calculate one gradient descent step for each weight
        # Step 1: Calculate the inputs x weights (only one dataset entry, no sum required)
        h = np.dot(x, weights)

        # Step 2: Calculate output of neural network
        nn_output = sigmoid(h)

        # Step 3: Calculate error of neural network
        error = y - nn_output

        # Step 4: Calculate the error term (intermediate step to calculate the gradient)
        # It requires the activation function derivate - for this case activation function
        # sigmoid_prime = sigmoid(h) * (1 - sigmoid(h))
        error_term = error * sigmoid(h) * (1 - sigmoid(h))

        # Step 5: Calculate change in weights
        del_w += learnrate * error_term * x

    # Update weights
    weights += del_w / n_records

    # Printing out the mean square error on the training set
    if e % (epochs / 10) == 0:
        out = sigmoid(np.dot(features, weights))
        loss = np.mean((out - targets) ** 2)
        if last_loss and last_loss < loss:
            print("Train loss: ", loss, "  WARNING - Loss Increasing")
        else:
            print("Train loss: ", loss)
        last_loss = loss


# Calculate accuracy on test data
tes_out = sigmoid(np.dot(features_test, weights))
predictions = tes_out > 0.5
accuracy = np.mean(predictions == targets_test)
print("Prediction accuracy: {:.3f}".format(accuracy))

('Train loss: ', 0.26404323469906082)
('Train loss: ', 0.2516071930665722)
('Train loss: ', 0.24211081845113358)
('Train loss: ', 0.23463525567444132)
('Train loss: ', 0.22862519972113274)
('Train loss: ', 0.22373253255332445)
('Train loss: ', 0.21972023929981527)
('Train loss: ', 0.21641376247182467)
('Train loss: ', 0.2136776976696495)
('Train loss: ', 0.21140425448119946)
Prediction accuracy: 0.775
